# 6.Método Cadena Markov-Monte Carlo (MCMC)

In [46]:
using LinearAlgebra
using BenchmarkTools
using SparseArrays
using Random
using Base.Threads: @threads,@spawn
using Plots
using PyCall
using DataFrames

## Implementación de la función que construye el sistema de ecuaciones de prueba $Ax = b$

In [47]:
function matriz_dispersa(n)
    e = ones(n)
    n2 =Int(n/2)
    diags = [-1,0,1]
    A = Matrix(spdiagm(-1 => -ones(n-1)
        ,0 => 3*ones(n),1 => -ones(n-1)))
    c = spdiagm(0 => ones(n)/2)
    ab = [x for x=1:n]
    ba = [(n+1)-x for x=1:n]
    c = Matrix(permute(c, ba, ab))

    A = A + c
    A[n2+1,n2] = -1
    A[n2,n2+1] = -1
    
    b = zeros(n,1)
    b[1] = 2.5
    b[n] = 2.5
    b[2:n-1] .= [1.5]
    b[n2:n2+1] .= [1]
    return A,b
end

matriz_dispersa (generic function with 1 method)

### Evaluación de parámetros

In [48]:
function preparametros(A,b,ϵ,δ)
    M = diagm(0 => diag(A))
    N = M-A
    T = inv(M) * N
    f = inv(M) * b
    nT, mT = size(T);
    print(nT, mT )

    S = fill(0, nT)
    P = fill(0., nT, mT)
    Pa = P
    [S[i] += 1 for i in 1:nT, j in 1:mT if T[i,j] != 0]
    [P[i,j]= 1/S[i] for i in 1:nT, j in 1:mT if T[i,j] != 0 ]
    Pa = [accumulate(+, P[i, 1:mT]) for i in 1:nT]
    Pi = [1/nT for i in 1:nT];
    Nc = floor((0.6745/δ)^2*((norm(f)^2)/(1-norm(T))^2)) + 1
    return Nc, mT, f, Pa, T, P
end

preparametros (generic function with 1 method)

### Matriz de probabilidad acumulada pre-build con envío de parámetros, type-anotations y view

In [95]:
function mcmc_acc_par_ta_thread(ϵ, Nc, mT, f, Pa, T, P)
    Xs = fill(0., mT)
        
    for i in 1:mT
        W_0 = 1.0
        for s in 1:Nc
            W = W_0; point = i; X = W_0 * f[i]::Float64
            while abs(W) >= ϵ
                nextpoint  = 1::Int64
                u = rand()
                while u >= Pa[point][nextpoint]::Float64
                    nextpoint = nextpoint + 1::Int64
                end
                    W_new = W *(T[point, nextpoint]/P[point, nextpoint])::Float64
                    X = X + W_new * f[nextpoint]::Float64
                point = nextpoint::Int64
                W = W_new::Float64
            end
        Xs[i] += X::Float64
        end
    end
        
    Xs = Xs/Nc::Float64
end

mcmc_acc_par_ta_thread (generic function with 1 method)

In [96]:
function mcmc_acc_par_ta_spawn(ϵ, Nc, mT, f, Pa, T, P)
    Xs = fill(0., mT)
    Xss = Threads.Atomic{Float64}(1)
    @sync begin
        
        for i in 1:mT
            W_0 = 1.0
            Threads.@spawn begin
                for s in 1:Nc
                    W = W_0; point = i; X = W_0 * f[i]::Float64
                    while abs(W) >= ϵ
                        nextpoint  = 1::Int64
                        u = rand()
                        while u >= Pa[point][nextpoint]::Float64
                            nextpoint = nextpoint + 1::Int64
                        end
                            W_new = W *(T[point, nextpoint]/P[point, nextpoint])::Float64
                             X = X + W_new * f[nextpoint]::Float64
                        point = nextpoint::Int64
                        W = W_new::Float64
                    end
                Xs[i] += X::Float64
                end
                
            end
        end
        Xss = Xs[]/Nc::Float64       
    end

end

mcmc_acc_par_ta_spawn (generic function with 1 method)

In [77]:
JULIA_NUM_THREADS=4
Threads.nthreads()

1

In [74]:
Threads.nthreads()

1

In [97]:
n = [6, 10, 30, 50]
A,b = matriz_dispersa(n[1])
ϵ = 0.1
δ = 0.1 
(Nc, mT, f, Pa, T, P) = preparametros(A,b,ϵ,δ)


66

(8623.0, 6, [0.8333333333333333; 0.5; … ; 0.5; 0.8333333333333333], [[0.0, 0.5, 0.5, 0.5, 0.5, 1.0], [0.3333333333333333, 0.3333333333333333, 0.6666666666666666, 0.6666666666666666, 1.0, 1.0], [0.0, 0.5, 0.5, 1.0, 1.0, 1.0], [0.0, 0.0, 0.5, 0.5, 1.0, 1.0], [0.0, 0.3333333333333333, 0.3333333333333333, 0.6666666666666666, 0.6666666666666666, 1.0], [0.5, 0.5, 0.5, 0.5, 1.0, 1.0]], [0.0 0.3333333333333333 … 0.0 -0.16666666666666666; 0.3333333333333333 0.0 … -0.16666666666666666 0.0; … ; 0.0 -0.16666666666666666 … 0.0 0.3333333333333333; -0.16666666666666666 0.0 … 0.3333333333333333 0.0], [0.0 0.5 … 0.0 0.5; 0.3333333333333333 0.0 … 0.3333333333333333 0.0; … ; 0.0 0.3333333333333333 … 0.0 0.3333333333333333; 0.5 0.0 … 0.5 0.0])

In [57]:
@benchmark mcmc_acc_par_ta_thread($ϵ, $Nc, $mT, $f, $Pa, $T, $P)

BenchmarkTools.Trial: 830 samples with 1 evaluation.
 Range (min … max):  5.371 ms …   8.340 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     5.928 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.019 ms ± 566.439 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▄█▂▁                                                         
  ████▇▇▅▅▄▅▆▆▆▄▆█▅▅▇▆▆▅▅▅▅▆▄▄▄▄▃▃▄▃▄▃▃▃▃▃▁▂▂▂▂▃▂▃▂▂▁▂▂▂▂▃▁▂▂ ▃
  5.37 ms         Histogram: frequency by time        7.88 ms <

 Memory estimate: 256 bytes, allocs estimate: 2.

In [98]:
@benchmark mcmc_acc_par_ta_spawn($ϵ, $Nc, $mT, $f, $Pa, $T, $P)

BenchmarkTools.Trial: 280 samples with 1 evaluation.
 Range (min … max):  16.618 ms … 22.393 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     17.263 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   17.862 ms ±  1.196 ms  ┊ GC (mean ± σ):  3.35% ± 5.06%

  ▁ ▄█                                                         
  █████▆▇▆▃▂▄▄▄▂▄▂▄▃▄▄▄▃▃▄▅▆▄▆▆▄▃▂▃▂▃▂▂▁▂▃▁▁▃▂▃▁▃▂▂▁▁▁▁▁▁▁▁▁▂ ▃
  16.6 ms         Histogram: frequency by time        21.8 ms <

 Memory estimate: 11.99 MiB, allocs estimate: 785440.

In [59]:

X  = @btime mcmc_acc_par_ta_thread($ϵ, $Nc, $mT, $f, $Pa, $T, $P)


  5.372 ms (2 allocations: 256 bytes)


6-element Array{Float64,1}:
 0.9950365023709914
 1.0130133387855134
 1.0038897709844548
 1.0043856216355755
 0.980510090979655
 0.9983028807538772

In [99]:
Xs  = @btime mcmc_acc_par_ta_spawn($ϵ, $Nc, $mT, $f, $Pa, $T, $P)

  16.582 ms (786002 allocations: 12.00 MiB)


6-element Array{Float64,1}:
 8775.44187242786
 8674.236596936427
 8644.43987197075
 8618.90512117059
 8654.484610768133
 8521.900205761258

In [88]:
norm(b-A*X)

0.07574668702828037

In [64]:
norm(b-A*Xs)

37659.6123360787